# AllSides sources & bias crawler

Get and save a list of rated news sources as left or right and in between.

A CSV file will be created with the following columns:

- Source
- Label
- Agree
- Disagree
- Publisher URL
- Publisher site

In [1]:
!ipython -m pip install aiohttp bs4 requests

]0;IPython: research/crawlersRequirement already satisfied: aiohttp in /Users/cmin/anaconda3/lib/python3.7/site-packages (3.6.2)


In [2]:
import asyncio
import csv
import logging
import re
import urllib.parse as urlparse

import aiohttp

import bs4
import requests

In [3]:
url_tpl = "https://www.allsides.com/media-bias/media-bias-ratings?field_featured_bias_rating_value=All&field_news_source_type_tid%5B1%5D=1&field_news_source_type_tid%5B2%5D=2&field_news_source_type_tid%5B3%5D=3&field_news_bias_nid_1%5B1%5D=1&field_news_bias_nid_1%5B2%5D=2&field_news_bias_nid_1%5B3%5D=3&title=&customFilter=1&page={}"
html_parser = "html5lib"
csv_header = [
    "source",
    "label",
    "agree",
    "disagree",
    "publisher",
    "site",
]
dump_path = "media-bias.csv"
encoding = "utf-8"
skip_blocked_sites = True

verbose = True  # make it True to see debugging messages
level = logging.DEBUG if verbose else logging.INFO
logging.root.handlers.clear()
logging.basicConfig(
    format="%(levelname)s - %(name)s - %(asctime)s - %(message)s",
    level=level
)

site_adapter = {
    "www.huffingtonpost.com": "www.huffpost.com",
    "www.cnn.com": "edition.cnn.com",
}


async def get_soup(session, url):
    abs_url = urlparse.urljoin(url_tpl, url)
    text = await (await session.get(abs_url)).text()
#     resp.raise_for_status()
    soup = bs4.BeautifulSoup(text, html_parser)
    return soup


def _adapt_site(url, netloc):
    site = site_adapter.get(netloc)
    if site:
        url = url.replace(netloc, site)
        netloc = site
    return url, netloc


async def get_publisher_url(session, src_url, source_name):
#     import code; code.interact(local={**globals(), **locals()})
    logging.debug("Getting publisher's URL for %r.", source_name)
    soup = await get_soup(session, src_url)
    div = soup.find("div", class_="dynamic-grid")
    if not div:
        return None
    
    url = div.find("a").get("href").strip()
    parsed = urlparse.urlparse(url)
    if not parsed.netloc:
        return None
    
    return _adapt_site(url, parsed.netloc)


async def save_pages(bias_writer, csvfile):
    async with aiohttp.ClientSession() as session:
        page = 0  # custom page if you want
        while True:
            logging.info("Crawling page %d...", page)
            url = url_tpl.format(page)
            soup = await get_soup(session, url)
            
            pub_coros = []
            extras = []
            table = soup.find("table")
            if not table or "no record" in table.find("tbody").find("tr").text.lower():
                logging.info("Reached empty table -> end of results/pages.")
                break
                
            for row in table.find("tbody").find_all("tr"):
                src_a = row.find("td", class_="source-title").find("a")
                src_url = src_a.get("href")
                source_name = src_a.text
                label_alt = row.find("td", class_="views-field-field-bias-image").find("img").get("alt")
                label = label_alt.split(":")[-1].strip()
                feedback = row.find("td", class_="community-feedback")
                agree = int(feedback.find("span", class_="agree").text)
                disagree = int(feedback.find("span", class_="disagree").text)
                
                extras.append([source_name, label, agree, disagree])
#                 import code; code.interact(local={**globals(), **locals()})
                pub_coros.append(get_publisher_url(session, src_url, source_name))
            
            publisher_details_list = await asyncio.gather(*pub_coros)
            for idx, publisher_details in enumerate(publisher_details_list):
                if not publisher_details:
                    if skip_blocked_sites:
                        continue
                    else:
                        publisher_details = ("", "")

#                 print(source_name, label, f"{agree}/{disagree}")
                bias_writer.writerow(extras[idx] + list(publisher_details))

            page += 1
            csvfile.flush()


async def main():
    with open(dump_path, "w", newline="", encoding=encoding) as csvfile:
        bias_writer = csv.writer(csvfile)
        bias_writer.writerow(csv_header)
        await save_pages(bias_writer, csvfile)
        
        
await main()

INFO - root - 2020-03-25 13:51:45,447 - Crawling page 0...
DEBUG - root - 2020-03-25 13:51:50,189 - Getting publisher's URL for 'AARP'.
DEBUG - root - 2020-03-25 13:51:50,191 - Getting publisher's URL for 'ABC News'.
DEBUG - root - 2020-03-25 13:51:50,225 - Getting publisher's URL for 'Abridge News'.
DEBUG - root - 2020-03-25 13:51:50,226 - Getting publisher's URL for 'Accuracy in Media '.
DEBUG - root - 2020-03-25 13:51:50,227 - Getting publisher's URL for 'ACLU'.
DEBUG - root - 2020-03-25 13:51:50,229 - Getting publisher's URL for 'AJ+'.
DEBUG - root - 2020-03-25 13:51:50,232 - Getting publisher's URL for 'Al Cardenas'.
DEBUG - root - 2020-03-25 13:51:50,233 - Getting publisher's URL for 'Al Jazeera'.
DEBUG - root - 2020-03-25 13:51:50,235 - Getting publisher's URL for 'AllSides'.
DEBUG - root - 2020-03-25 13:51:50,236 - Getting publisher's URL for 'AllSides Community'.
DEBUG - root - 2020-03-25 13:51:50,243 - Getting publisher's URL for 'Allysia Finley (Wall Street Journal)'.
DEBUG 

INFO - root - 2020-03-25 13:52:09,552 - Crawling page 3...
DEBUG - root - 2020-03-25 13:52:13,045 - Getting publisher's URL for 'Chris Britt (cartoonist)'.
DEBUG - root - 2020-03-25 13:52:13,047 - Getting publisher's URL for 'Chris Ruddy'.
DEBUG - root - 2020-03-25 13:52:13,048 - Getting publisher's URL for 'Christian Science Monitor'.
DEBUG - root - 2020-03-25 13:52:13,049 - Getting publisher's URL for 'Christiane Amanpour'.
DEBUG - root - 2020-03-25 13:52:13,053 - Getting publisher's URL for 'Christianity Today'.
DEBUG - root - 2020-03-25 13:52:13,056 - Getting publisher's URL for 'Christopher Buskirk'.
DEBUG - root - 2020-03-25 13:52:13,058 - Getting publisher's URL for 'Chuck Asay (cartoonist)'.
DEBUG - root - 2020-03-25 13:52:13,060 - Getting publisher's URL for 'City Journal'.
DEBUG - root - 2020-03-25 13:52:13,064 - Getting publisher's URL for 'CivilPolitics.org'.
DEBUG - root - 2020-03-25 13:52:13,070 - Getting publisher's URL for 'Clay Bennett (cartoonist)'.
DEBUG - root - 202

DEBUG - root - 2020-03-25 13:52:33,821 - Getting publisher's URL for 'Forbes'.
DEBUG - root - 2020-03-25 13:52:33,822 - Getting publisher's URL for 'Foreign Affairs'.
DEBUG - root - 2020-03-25 13:52:33,823 - Getting publisher's URL for 'Foreign Policy'.
DEBUG - root - 2020-03-25 13:52:33,828 - Getting publisher's URL for 'Fox News Latino'.
DEBUG - root - 2020-03-25 13:52:33,830 - Getting publisher's URL for 'Fox News Opinion'.
DEBUG - root - 2020-03-25 13:52:33,832 - Getting publisher's URL for 'Fox Online News'.
DEBUG - root - 2020-03-25 13:52:33,835 - Getting publisher's URL for 'Frank Bruni'.
DEBUG - root - 2020-03-25 13:52:33,837 - Getting publisher's URL for 'Frank Miniter'.
DEBUG - root - 2020-03-25 13:52:33,842 - Getting publisher's URL for 'Freakonomics'.
DEBUG - root - 2020-03-25 13:52:33,844 - Getting publisher's URL for 'Freedom House'.
DEBUG - root - 2020-03-25 13:52:33,846 - Getting publisher's URL for 'Froma Harrop'.
DEBUG - root - 2020-03-25 13:52:33,848 - Getting publis

DEBUG - root - 2020-03-25 13:52:52,648 - Getting publisher's URL for 'Jonathan Miller'.
DEBUG - root - 2020-03-25 13:52:52,650 - Getting publisher's URL for "Journalist's Resource".
DEBUG - root - 2020-03-25 13:52:52,652 - Getting publisher's URL for 'Juan Williams'.
DEBUG - root - 2020-03-25 13:52:52,654 - Getting publisher's URL for 'Jubilee Media'.
DEBUG - root - 2020-03-25 13:52:52,656 - Getting publisher's URL for 'Judicial Watch'.
DEBUG - root - 2020-03-25 13:52:52,660 - Getting publisher's URL for 'Judy Sheindlin'.
DEBUG - root - 2020-03-25 13:52:52,668 - Getting publisher's URL for 'Julian Zelizer'.
DEBUG - root - 2020-03-25 13:52:52,673 - Getting publisher's URL for 'Kathleen Parker'.
DEBUG - root - 2020-03-25 13:52:52,677 - Getting publisher's URL for 'Katie Hill'.
DEBUG - root - 2020-03-25 13:52:52,680 - Getting publisher's URL for 'KATU'.
DEBUG - root - 2020-03-25 13:52:52,683 - Getting publisher's URL for 'Ken Catalino (cartoonist)'.
DEBUG - root - 2020-03-25 13:52:52,684 

DEBUG - root - 2020-03-25 13:53:13,739 - Getting publisher's URL for 'New York Magazine'.
DEBUG - root - 2020-03-25 13:53:13,741 - Getting publisher's URL for 'New York Post'.
DEBUG - root - 2020-03-25 13:53:13,744 - Getting publisher's URL for 'New York Times - News'.
DEBUG - root - 2020-03-25 13:53:13,751 - Getting publisher's URL for 'New York Times - Opinion'.
DEBUG - root - 2020-03-25 13:53:13,754 - Getting publisher's URL for 'NewsBusters'.
DEBUG - root - 2020-03-25 13:53:13,756 - Getting publisher's URL for 'Newsmax - News'.
DEBUG - root - 2020-03-25 13:53:13,758 - Getting publisher's URL for 'Newsmax - Opinion'.
DEBUG - root - 2020-03-25 13:53:13,760 - Getting publisher's URL for 'NewsOne'.
DEBUG - root - 2020-03-25 13:53:13,761 - Getting publisher's URL for 'Newsweek'.
DEBUG - root - 2020-03-25 13:53:13,762 - Getting publisher's URL for 'Newt Gingrich'.
DEBUG - root - 2020-03-25 13:53:13,764 - Getting publisher's URL for 'Nicholas Kristof'.
DEBUG - root - 2020-03-25 13:53:13,7

DEBUG - root - 2020-03-25 13:53:35,494 - Getting publisher's URL for 'Ryan Cooper'.
DEBUG - root - 2020-03-25 13:53:35,496 - Getting publisher's URL for 'S.E. Cupp'.
DEBUG - root - 2020-03-25 13:53:35,498 - Getting publisher's URL for 'Saagar Enjeti'.
DEBUG - root - 2020-03-25 13:53:35,502 - Getting publisher's URL for 'Sally Pipes'.
DEBUG - root - 2020-03-25 13:53:35,505 - Getting publisher's URL for 'Salon'.
DEBUG - root - 2020-03-25 13:53:35,507 - Getting publisher's URL for 'San Francisco Chronicle'.
DEBUG - root - 2020-03-25 13:53:35,510 - Getting publisher's URL for 'San Jose Mercury News'.
DEBUG - root - 2020-03-25 13:53:35,514 - Getting publisher's URL for 'SBG'.
DEBUG - root - 2020-03-25 13:53:35,517 - Getting publisher's URL for 'Science Daily'.
DEBUG - root - 2020-03-25 13:53:35,519 - Getting publisher's URL for 'Scientific American'.
DEBUG - root - 2020-03-25 13:53:35,522 - Getting publisher's URL for 'Scott Adams'.
DEBUG - root - 2020-03-25 13:53:35,523 - Getting publisher

DEBUG - root - 2020-03-25 13:53:55,318 - Getting publisher's URL for 'The Saturday Evening Post'.
DEBUG - root - 2020-03-25 13:53:55,320 - Getting publisher's URL for 'The Telegraph - UK'.
DEBUG - root - 2020-03-25 13:53:55,322 - Getting publisher's URL for 'The Texan'.
DEBUG - root - 2020-03-25 13:53:55,326 - Getting publisher's URL for 'The Texas Observer'.
DEBUG - root - 2020-03-25 13:53:55,329 - Getting publisher's URL for 'The Texas Tribune'.
DEBUG - root - 2020-03-25 13:53:55,332 - Getting publisher's URL for 'The Verge'.
DEBUG - root - 2020-03-25 13:53:55,334 - Getting publisher's URL for 'The Week - News'.
DEBUG - root - 2020-03-25 13:53:55,337 - Getting publisher's URL for 'The Week - Opinion'.
DEBUG - root - 2020-03-25 13:53:55,341 - Getting publisher's URL for 'The Weekly Standard'.
DEBUG - root - 2020-03-25 13:53:55,345 - Getting publisher's URL for 'The Western Journal'.
DEBUG - root - 2020-03-25 13:53:55,350 - Getting publisher's URL for 'TheBlaze.com'.
DEBUG - root - 202

Some publishers are blocked (no websites offered by AllSides), therefore fewer results in the CSV file.

Now let's find a good way of associating a side with a website in case multiple candidates are available.

In [4]:
side_dict = {}

with open(dump_path, newline="") as stream:
    reader = csv.reader(stream)
    print(next(reader))
    
    for row in reader:
        side_dict.setdefault(row[5], []).append((row[0], row[1], row[2]))

for site, sides in side_dict.items():
    if len(sides) > 1:
        print(site, sides)

['source', 'label', 'agree', 'disagree', 'publisher', 'site']
edition.cnn.com [('CNN (Web News)', 'Lean Left', '30618'), ('CNN - Editorial', 'Left', '11869')]
www.thedailybeast.com [('Daily Beast', 'Left', '9681'), ('Newsweek', 'Lean Left', '2253')]
www.foxnews.com [('Fox News Opinion', 'Right', '9842'), ('Fox Online News', 'Lean Right', '25701')]
www.courier-journal.com [('Louisville Courier-Journal', 'Lean Left', '198'), ('The Courier-Journal', 'Lean Left', '134')]
www.nytimes.com [('New York Times - News', 'Lean Left', '17834'), ('New York Times - Opinion', 'Left', '4372')]
www.newsmax.com [('Newsmax - News', 'Lean Right', '7017'), ('Newsmax - Opinion', 'Right', '56')]
www.npr.org [('NPR Editorial ', 'Lean Left', '2732'), ('NPR Online News', 'Center', '21145')]
theweek.com [('The Week - News', 'Center', '3207'), ('The Week - Opinion', 'Lean Left', '37')]
online.wsj.com [('Wall Street Journal - Editorial', 'Lean Right', '5916'), ('Wall Street Journal - News', 'Center', '13807')]
